In [ ]:
%reload_ext autoreload
%autoreload 
%matplotlib inline

In [ ]:
# !sudo apt install tesseract-ocr
# !pip install pytesseract
# ! pip install vaderSentiment
# !pip install flair

In [ ]:
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow
import os
import pandas as pd
import re
import nltk 
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import flair

import pytesseract
from PIL import Image,ImageOps
import numpy as np
import re
from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = '/content/gdrive/My Drive/app/data/Text_Det'

Mounted at /content/gdrive


In [ ]:
#Method 1 to determine the text on image

def GetImageText3(img, thres):
  img = Image.open(img).convert("L")
  img = ImageOps.invert(img)

  threshold = thres
  table = []
  pixelArray = img.load()
  for y in range(img.size[1]):
    List = []
    
    for x in range(img.size[0]):
      if pixelArray[x,y] < threshold:
        List.append(0)
      else:
        List.append(255)
    table.append(List)

  img = Image.fromarray(np.array(table,dtype="uint8")) # load the image from array.
  txt = pytesseract.image_to_string(img)
  
  return (txt)

In [1]:
def Find_Image(img):
  
  str_final =""

  for i in range(10, 50, 10):
    txt= GetImageText3(img,i)

    txt_len = len(txt)

    if (len(txt)>0):
      if (len(txt)>len(str_final)):
        str_final = txt

  return str_final

In [ ]:
#Method 2 to determine the text on image

def GetImageText1(img_name):
  
  str_final =""
  found=0

  # Load image, grayscale, Gaussian blur, adaptive threshold
  image = cv2.imread(img_name)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (9,9), 0)
  thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

  # Dilate to combine adjacent text contours
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
  dilate = cv2.dilate(thresh, kernel, iterations=4)

  # Find contours, highlight text areas, and extract ROIs
  cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]


  ROI_number = 0
  for c in cnts:
    area = cv2.contourArea(c)
    if area > 10:
      found=1
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)
      text = image[y:y+h, x:x+w]
        
  custom_config = r'-l eng --oem 3 --psm 6'

  #cv2_imshow( text)
  if found!=0:
    str_final=pytesseract.image_to_string(text,config=custom_config)

  return (str_final)


In [ ]:
#Method 3 to determine the text on image

def GetImageText2(img_name):

  str_final =""
  found=0

  img = cv2.imread(img_name)

  # convert to gray
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # threshold image
  thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)[1]

  # apply morphology to clean up small white or black regions
  kernel = np.ones((5,5), np.uint8)
  morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
  morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

  # thin region to remove excess black border
  kernel = np.ones((3,3), np.uint8)
  morph = cv2.morphologyEx(morph, cv2.MORPH_ERODE, kernel)

  # find contours
  cntrs = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]

  # Contour filtering -- keep largest, vertically oriented object (h/w > 1)
  area_thresh = 10
  for c in cntrs:
    area = cv2.contourArea(c)
    x,y,w,h = cv2.boundingRect(c)
    aspect = h / w
    if area > area_thresh:
      found = 1
      big_contour = c
      area_thresh = area

 

  if(found==1):
    x,y,w,h = cv2.boundingRect(big_contour)
    text = img[y:y+h, x:x+w]
    custom_config = r'-l eng --oem 3 --psm 6'
    str_final= pytesseract.image_to_string(text,config=custom_config)

  return (str_final)


In [ ]:
_, _, files = next(os.walk(base_dir))
file_count = len(files)
print(file_count)

239


In [ ]:
df_final = pd.DataFrame(columns=['image', 'txt1', 'txt2','txt3'])

In [ ]:
i = 0
for filename in os.listdir(base_dir):

  try:
    str1= GetImageText1(os.path.join(base_dir, filename))
    str2= GetImageText2(os.path.join(base_dir, filename))
    str3= Find_Image(os.path.join(base_dir, filename))
    
    df_final.loc[i] =[filename,str1,str2,str3]
    i=i+1
  except Exception as e:
    print(filename)

In [ ]:
df_final.head(5)
df_final_bak= df_final.copy()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
############## Text Cleanup ###################
stopword = nltk.corpus.stopwords.words('english')

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
 
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
stop = stopwords.words('english')

In [ ]:
df_final['final_txt'] = np.select([len(df_final.txt1) > 0 , len(df_final.txt2) > 0], [df_final.txt1, df_final.txt2], default="")
df_final['final_txt'] = np.select([len(df_final.final_txt) > 0 , len(df_final.txt3) > 0], [df_final.final_txt, df_final.txt3], default="")

In [ ]:
df_final['final_txt']= df_final['final_txt'].apply(lambda x: remove_punct(x))
df_final['final_txt']  = df_final['final_txt'] .apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df_final['final_txt'] =df_final['final_txt'].apply(lambda x: tokenization(x.lower()))

#df_final['final_txt']  =df_final['final_txt'] .apply(lambda x: stemming(x))
#df_final['final_txt']  = df_final['final_txt'].apply(lambda x: lemmatizer(x))

df_final['final_txt']= df_final['final_txt'].apply(lambda x: re.sub("\s\s+" , " ", x))

In [ ]:
analyser = SentimentIntensityAnalyzer()

In [ ]:
df_final["Sentiment"]="Random"

In [ ]:
#Find the sentiment of each text found 
for i,row in df_final.iterrows():

  str_final ="Positive"

  if (len(row['txt1'])>0):
    
    sent= analyser.polarity_scores(row['txt1'])['compound']
  
    if (sent>= 0.05):
      str_final="Positive"
    elif (sent>-0.05 and sent <0.05):
      str_final= "Random"
    else:
      str_final= "Negative"
     
    df_final.at[i,'Sentiment'] = str_final

In [ ]:
df_final1=df_final[["image","Sentiment"]]
df_final1.columns = ['Filename', 'Category']

In [ ]:
df_final1.to_csv("/content/gdrive/My Drive/app/data/Image_out.csv", index= False)

In [ ]:
df_final1.shape

(239, 2)

In [ ]:
########### Using Flair

In [ ]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
for i,row in df_final.iterrows():

  str_final ="Positive"

  if (len(row['final_txt'])>0):
    
    s = flair.data.Sentence(row['final_txt'])
    flair_sentiment.predict(s)
    total_sentiment =str( s.labels)
    
    if ("POSITIVE" in total_sentiment):
      str_final="Positive"
    elif ("NEGATIVE" in total_sentiment):
      str_final= "Negative"
    else:
      str_final= "Random"
     
    df_final.at[i,'Sentiment'] = str_final
  

2020-07-16 14:39:21,153 loading file /root/.flair/models/sentiment-en-mix-distillbert.pt


In [ ]:
df_final.Sentiment.value_counts()

Random      154
Positive     47
Negative     38
Name: Sentiment, dtype: int64

In [ ]:
#Using Text Blob
def detect_sent(text):
  str_final="Random"
  
  pol =TextBlob(text).sentiment.polarity
 
  if (pol>= 0.1):
    str_final="Positive"
  elif (pol>-0.1 and pol <0.1):
      str_final= "Random"
  else:
      str_final= "Negative"

  return (str_final)
df_final['Sentiment'] = df_final.final_txt.apply(detect_sent)
df_final.head()

,image,txt1,txt2,txt3,final_txt,polarity,Sentiment
0,Test2434.jpg,,‘NOW. IN THE MANU-SAMMITA IT 1S CLEARLY STATED...,nba hinade wehameni‘lide La com hihi\nLr Llsmm...,,Random,Random
1,Test145.jpg,,ofl\na)\nee,,,Random,Random
2,Test230.jpg,Top Pro 8 C\nQuotes,66\n55\nTop Pro & Con\nQuotes,Top Pro & Con\nQuotes,Top Pro C Quotes,Positive,Positive
3,Test2959.jpg,,"La\n,",,,Random,Random
4,Test109.jpg,,>\n5,,,Random,Random


In [ ]:
df_final=df_final[["image","Sentiment"]]
df_final.columns = ['Filename', 'Category']

In [ ]:
df_final.to_csv("/content/gdrive/My Drive/app/data/text.csv", index= False)